# 计算冰点和沸点距离现在的天数


In [51]:
import pandas as pd
import datetime as dt


In [52]:
def create_hot_ice(data):

        daily_data = pd.DataFrame(columns=["日期", "板块名称", "沸点", '冰点'])
        grouped_data = data.groupby("板块名称")

        # 遍历每个板块
        # 按照 date 和 sector 两个字段进行分组
        grouped_data = data.groupby(["日期", "板块名称"])

        # 遍历每个分组
        for (date, sector), group_data in grouped_data:
            # 获取当日所有股票的涨跌幅数据
            # 获取该分组中股票的涨跌幅情况
            day_data = group_data["涨跌幅"]
            # day_data = sector_data.loc[sector_data["日期"] == date, "涨跌幅"]

            # 判断当日的涨跌幅情况是否全为正数，全为负数，还是既有正数又有负数
            if all(day_data > 0):
                hot = 1
                cold = 0
            elif all(day_data < 0):
                hot = 0
                cold = 1
            else:
                hot = 0
                cold = 0

            # 将结果添加到 daily_data 中
            # daily_data = daily_data.append(
            #     {"日期": date, "板块名称": sector, "沸点": hot, "冰点": cold}, ignore_index=True)
            daily_data = pd.concat([daily_data, pd.DataFrame(
                {"日期": date, "板块名称": sector, "沸点": hot, "冰点": cold}, index=[0])], ignore_index=True)
        # 将 daily_data 合并回原始 DataFrame 中
        # data = pd.merge(data, daily_data, on=["日期", "板块名称"])
        return daily_data


In [53]:


# 获取历史数据
data = pd.read_csv("../data/Hist_2023-03-28.csv", index_col=0,
                   parse_dates=True, dtype={"股票代码": object})

data.head()


,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,板块名称,股票代码
日期,,,,,,,,,,,,
2023-03-01,22.55,22.42,22.83,22.18,98735,221958533.0,2.90,0.18,0.04,3.03,动物保健,300119
2023-03-02,22.44,21.73,22.54,21.64,53425,117040504.0,4.01,-3.08,-0.69,1.64,动物保健,300119
2023-03-03,21.89,21.27,22.18,21.12,57008,122635566.0,4.88,-2.12,-0.46,1.75,动物保健,300119
2023-03-06,21.30,21.73,21.89,21.17,49037,106350983.0,3.39,2.16,0.46,1.50,动物保健,300119
2023-03-07,21.82,21.27,22.22,21.17,41433,89342602.0,4.83,-2.12,-0.46,1.27,动物保健,300119


In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 96597 entries, 2023-03-01 to 2023-03-27
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   开盘      96597 non-null  float64
 1   收盘      96597 non-null  float64
 2   最高      96597 non-null  float64
 3   最低      96597 non-null  float64
 4   成交量     96597 non-null  int64  
 5   成交额     96597 non-null  float64
 6   振幅      96597 non-null  float64
 7   涨跌幅     96597 non-null  float64
 8   涨跌额     96597 non-null  float64
 9   换手率     96597 non-null  float64
 10  板块名称    96597 non-null  object 
 11  股票代码    96597 non-null  object 
dtypes: float64(9), int64(1), object(2)
memory usage: 9.6+ MB


## 标记沸点和冰点的日期

In [55]:
data.reset_index(inplace=True)

In [62]:
result=create_hot_ice(data)
result.head()

,日期,板块名称,沸点,冰点
0,2023-03-01,IT服务,0,0
1,2023-03-01,LED,0,0
2,2023-03-01,专业工程,0,0
3,2023-03-01,专业服务,0,0
4,2023-03-01,专业连锁,0,0


In [57]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4465 entries, 0 to 4464
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   日期      4465 non-null   datetime64[ns]
 1   板块名称    4465 non-null   object        
 2   沸点      4465 non-null   object        
 3   冰点      4465 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 139.7+ KB


In [63]:
# result.set_index("日期",inplace=True)
# data=result.loc['2023-03-24']
result[(result['板块名称'] == "游戏") & (result['日期'] == "2023-03-24")]


,日期,板块名称,沸点,冰点
4136,2023-03-24,游戏,0,0


In [64]:
my = data.groupby(['日期', '板块名称'])['涨跌幅'].agg(
    [('涨的数量', lambda x: sum(x > 0)), ('跌的数量', lambda x: sum(x < 0)), ('平的数量', lambda x: sum(x == 0))])
my


涨的数量  跌的数量  平的数量
日期         板块名称                      
2023-03-01 IT服务       120     7     0
           LED         30     7     2
           专业工程        31     3     0
           专业服务        20     6     1
           专业连锁         6     1     1
...                   ...   ...   ...
2023-03-27 食品及饲料添加剂     6    13     1
           餐饮           2     1     0
           饰品           0    15     1
           饲料          11     7     0
           高速公路         0    20     0

[4465 rows x 3 columns]

In [65]:
my.loc[('2023-03-24','游戏')]


涨的数量    27
跌的数量     2
平的数量     1
Name: (2023-03-24 00:00:00, 游戏), dtype: int64

In [ ]:
data[data['板块名称'] == "游戏"]


In [ ]:
# 筛选出冰点或热点至少有一个为 True 的行
hot_or_cold = result[(result["沸点"] == True) | (result["冰点"] == True)]
hot_or_cold


In [ ]:
hot = data.groupby(["日期", "板块名称"]).apply(lambda x: all(x['涨跌幅'] > 0))
# 重置索引
hot = hot.reset_index()
ice = data.groupby(["日期", "板块名称"]).apply(lambda x: all(x['涨跌幅'] < 0))
ice = ice.reset_index()
ice.columns = ["日期", "板块名称", '冰点']
# 重命名列名
hot.columns = ["日期", "板块名称", '沸点']
result = pd.merge(ice, hot, on=["日期", "板块名称"])
result


# 计算沸点的天数

In [ ]:
df = result.copy()
# 找到冰点为true的日期
hot_dates = df.groupby("板块名称").apply(lambda x: x[x['沸点'] == True])[["日期","沸点"]]
# true_ice_dates.columns=["日期"]
# true_ice_dates = true_ice_dates.reset_index(drop=True).loc["日期", "板块名称"]
hot_dates.reset_index(inplace=True)

hot_dates=hot_dates[["日期", "板块名称"]]
hot_dates


## 计算冰点的天数

In [ ]:
df = result.copy()
# 找到冰点为true的日期
ice_dates = df.groupby("板块名称").apply(
    lambda x: x[x['冰点'] == True])[["日期", "冰点"]]
# true_ice_dates.columns=["日期"]
# true_ice_dates = true_ice_dates.reset_index(drop=True).loc["日期", "板块名称"]
ice_dates.reset_index(inplace=True)

ice_dates = ice_dates[["日期", "板块名称"]]
ice_dates


## 计算最近的冰点为true的天数

In [ ]:
# 计算最近的冰点为true的天数
def hot_date(specified_date, sect):
    dt_series = hot_dates[hot_dates['板块名称'] ==
                          sect]['日期'].apply(lambda x: x.date())
    # print(sect)
    time_list = dt_series.to_list()
    # print(time_list)
    if len(time_list) == 0:
        return 0
    # specified_date = specified_date.date()
    # 判断指定日期是否落在时间list中的区间
    diff_days = 0
    if specified_date in time_list:
        return -1
    if specified_date < time_list[0]:
        nearest_date = time_list[0]
        diff_days = 0
    
    elif specified_date > time_list[-1]:
        nearest_date = time_list[-1]
        diff_days = (specified_date-nearest_date).days
    else:
        for i in range(len(time_list) - 1):
            if time_list[i] <= specified_date < time_list[i+1]:
                nearest_date = time_list[i]
                diff_days = (specified_date-nearest_date).days
                break
    return diff_days


# data = result.groupby("板块名称").apply(
#     lambda row: 0 if row["冰点"] else 0)
mydata = []
grouped = result.groupby(["板块名称", "日期"])
for group_name, group_data in grouped:
    # print(type(group_name[1]))
    re = hot_date(group_name[1].date(), group_name[0], )
    mydata.append([group_name[0], group_name[1], re])
# data = result.groupby(["板块名称","日期"]).apply(
#     lambda row: 0 if row["冰点"].any() else mydate(row["日期"], row["板块名称"]))
# data = result.groupby(["板块名称","日期"]).apply(lambda row: 0 if row["冰点"].any() else 100)
# data
final_hot = pd.DataFrame(mydata)
final_hot.columns = ["板块名称", '日期', "距上次沸点天数"]
final_hot['日期'] = pd.to_datetime(final_hot['日期'])
final_hot.head()


In [ ]:
final_hot.info()


In [ ]:
result=pd.merge(result,final_hot,on=['日期','板块名称'])
result

In [ ]:
# 计算最近的冰点为true的天数
def ice_date(specified_date, sect):
    dt_series = ice_dates[ice_dates['板块名称'] ==
                          sect]['日期'].apply(lambda x: x.date())
    # print(sect)
    time_list = dt_series.to_list()
    # print(time_list)
    if len(time_list) == 0:
        return 0
    # specified_date = specified_date.date()
    # 判断指定日期是否落在时间list中的区间
    diff_days = 0
    if specified_date in time_list:
        return -1
    if specified_date < time_list[0]:
        nearest_date = time_list[0]
        diff_days = 0

    elif specified_date > time_list[-1]:
        nearest_date = time_list[-1]
        diff_days = (specified_date-nearest_date).days
    else:
        for i in range(len(time_list) - 1):
            if time_list[i] <= specified_date < time_list[i+1]:
                nearest_date = time_list[i]
                diff_days = (specified_date-nearest_date).days
                break
    return diff_days


# data = result.groupby("板块名称").apply(
#     lambda row: 0 if row["冰点"] else 0)
mydata = []
grouped = result.groupby(["板块名称", "日期"])
for group_name, group_data in grouped:
    # print(type(group_name[1]))
    re = ice_date(group_name[1].date(), group_name[0], )
    mydata.append([group_name[0], group_name[1], re])
# data = result.groupby(["板块名称","日期"]).apply(
#     lambda row: 0 if row["冰点"].any() else mydate(row["日期"], row["板块名称"]))
# data = result.groupby(["板块名称","日期"]).apply(lambda row: 0 if row["冰点"].any() else 100)
# data
ice_hot = pd.DataFrame(mydata)
ice_hot.columns = ["板块名称", '日期', "距上次冰点天数"]
ice_hot['日期'] = pd.to_datetime(ice_hot['日期'])
ice_hot.head()


In [ ]:
ice_hot.info()


In [ ]:
result = pd.merge(result, ice_hot, on=['日期', '板块名称'])
result


In [ ]:
result.to_csv("临时冰点沸点.csv",index=False)

## 检查结果

In [ ]:
result[result['板块名称'] == "IT服务"]


In [ ]:
result[result['板块名称'] == "化学制药"]


In [70]:
import pandas as pd

# 模拟股票数据
stock_data = pd.DataFrame({'Date': ['2023-03-27', '2023-03-27', '2023-03-27', '2023-03-28', '2023-03-28', '2023-03-28'],
                           'Sector': ['Technology', 'Technology', 'Finance', 'Technology', 'Finance', 'Finance'],
                           'Ticker': ['AAPL', 'GOOG', 'JPM', 'AAPL', 'JPM', 'C'],
                           'Return': [0.02, -0.01, 0.03, 0.01, -0.02, -0.01]})

grouped=stock_data.groupby(['Date', 'Sector'])
for name, group in grouped:
    print("Group Name: ", name)
    print(group['Return'])
    day_data = group['Return']
    if all(day_data > 0):
        hot = 1
        cold = 0
    elif all(day_data < 0):
        hot = 0
        cold = 1
    else:
        hot = 0
        cold = 0

    print(f"data is {hot} {cold}")



Group Name:  ('2023-03-27', 'Finance')
2    0.03
Name: Return, dtype: float64
data is 1 0
Group Name:  ('2023-03-27', 'Technology')
0    0.02
1   -0.01
Name: Return, dtype: float64
data is 0 0
Group Name:  ('2023-03-28', 'Finance')
4   -0.02
5   -0.01
Name: Return, dtype: float64
data is 0 1
Group Name:  ('2023-03-28', 'Technology')
3    0.01
Name: Return, dtype: float64
data is 1 0


In [72]:
hot = stock_data.groupby(["Date", "Sector"]).apply(
    lambda x: all(x['Return'] > 0))
hot

Date        Sector    
2023-03-27  Finance        True
            Technology    False
2023-03-28  Finance       False
            Technology     True
dtype: bool

In [74]:

# 重置索引
hot = hot.reset_index()
ice = stock_data.groupby(["日期", "板块名称"]).apply(lambda x: all(x['涨跌幅'] < 0))
ice = ice.reset_index()
ice.columns = ["日期", "板块名称", '冰点']
# 重命名列名
hot.columns = ["日期", "板块名称", '沸点']
result = pd.merge(ice, hot, on=["日期", "板块名称"])
result


KeyError: '日期'

In [68]:
# 按照日期和板块进行分组，并计算涨跌幅大于0和小于0的个数
result = stock_data.groupby(['Date', 'Sector'])['Return'].agg(
    [lambda x: (x > 0).sum(), lambda x: (x < 0).sum()])

# 重命名列名
result = result.rename(columns={
                       '<lambda_0>': 'Num_Positive_Returns', '<lambda_1>': 'Num_Negative_Returns'})

# 输出结果
print(result)


                       Num_Positive_Returns  Num_Negative_Returns
Date       Sector                                                
2023-03-27 Finance                        1                     0
           Technology                     1                     1
2023-03-28 Finance                        0                     2
           Technology                     1                     0


In [77]:
import plotly.express as px
import pandas as pd

# 读取 CSV 文件
# df = pd.read_csv('data.csv')
df = pd.DataFrame({'Date': ['2023-03-27', '2023-03-27', '2023-03-27', '2023-03-28', '2023-03-28', '2023-03-28'],
                           'Sector': ['Technology', 'Technology', 'Finance', 'Technology', 'Finance', 'Finance'],
                           'Ticker': ['AAPL', 'GOOG', 'JPM', 'AAPL', 'JPM', 'C'],
                           'Return': [0.02, -0.01, 0.03, 0.01, -0.02, -0.01]})
# 绘制双向条形图
fig = px.bar(df, x='Date', y='Sector', color='Ticker', orientation='h')

# 添加过滤器
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            buttons=list([
                dict(
                    label="All",
                    method="update",
                    args=[{"visible": [True, True, True]},
                          {"title": "All"}]
                ),
                dict(
                    label="Category A",
                    method="update",
                    args=[{"visible": [True, False, False]},
                          {"title": "Category A"}]
                ),
                dict(
                    label="Category B",
                    method="update",
                    args=[{"visible": [False, True, False]},
                          {"title": "Category B"}]
                ),
                dict(
                    label="Category C",
                    method="update",
                    args=[{"visible": [False, False, True]},
                          {"title": "Category C"}]
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        )
    ])

# 显示图表
fig.show()
